## makemore -- mlp

following andrej karpathy's course (https://www.youtube.com/watch?v=TCH_1BHY58I)

his github: https://github.com/karpathy/makemore

### high level

this is a *word* level language model, and has a defined vocabulary size of *n* words. Each word is represented with a vector embedding.

predict the next word given the previous word. works by tuning the embedding vectors over time.

# <img src="./images/mlp.png" width="500" style="margin: 0 auto; display: block;"/>

- predict next word using 3 previous words
- *n-1* indices
- lookup table *C* is a *n x d* matrix, where *d* is the embedding dimension (30 in this case) -> first layer has *3 x 30* neurons
- hidden layer: tanh - its a hyperparameter (design choice upto designer of neural net), can be as large or small as one wants, but each neuron here connected to all the neurons in the prev layer + next layer
- output layer: *n* neurons, one for each word in the vocabulary

In [1]:
!curl -O https://raw.githubusercontent.com/karpathy/makemore/refs/heads/master/names.txt
!pip install torch

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  222k  100  222k    0     0   381k      0 --:--:-- --:--:-- --:--:--  382k
  Using cached mpmath-1.3.0-py3-none-any.whl.metadata (8.6 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.5/74.5 MB 5.5 MB/s eta 0:00:0000:0100:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 19.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 13.3 MB/s eta 0:00:00a 0:00:01
Using cached mpmath-1.3.0-py3-none-any.whl (536 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6/6 [torch]32m5/6 [torch]]x]

[notice] A new release of pip is available: 25.1.1 -> 25.3
[notice] To update, run: /opt/homebrew/opt/python@3.11/bin/python3.11 -m pip install --upgrade pip


In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F

words = open('names.txt', 'r').read().splitlines()

In [4]:
# building the vocab

chars = sorted(list(set("".join(words))))
stoi = {c: i + 1 for i, c in enumerate(chars)}
stoi['.'] = 0
itos = {i: c for c, i in stoi.items()}
print(itos)


{1: 'a', 2: 'b', 3: 'c', 4: 'd', 5: 'e', 6: 'f', 7: 'g', 8: 'h', 9: 'i', 10: 'j', 11: 'k', 12: 'l', 13: 'm', 14: 'n', 15: 'o', 16: 'p', 17: 'q', 18: 'r', 19: 's', 20: 't', 21: 'u', 22: 'v', 23: 'w', 24: 'x', 25: 'y', 26: 'z', 0: '.'}


In [10]:
# building the dataset. taking three chars at a time, and assigning the output label as the fourth char

block_size = 3 # context length
X, Y = [], []

for word in words:
    context = [0] * block_size
    for ch in word + '.':
        ix = stoi[ch]
        X.append(context)
        Y.append(ix)
        context = context[1:] + [ix]

X = torch.tensor(X)
Y = torch.tensor(Y)

X.shape, Y.shape # each input to the neural net is 3 integers, and we have 228146 examples

(torch.Size([228146, 3]), torch.Size([228146]))

In [ ]:
C = torch.randn((27, 2)) # 27 chars in the vocab, and each word is represented with a 2 dimensional vector
